<a href="https://colab.research.google.com/github/guillermohenrion/AI/blob/master/text_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preprocesamiento de texto

## Observaciones generales

- símbolo extraño en encabezado
- encabezado: 
    - página pares: "ESTUDIOS PÚBLICOS"
    - páginas impares: "FRIEDRICH A. HAYEK"
- notas al pie: dos \t (?) y número de nota. En el texto aparece como un número

## 1. Carga del texto

In [ ]:
import re
import json
import string

In [ ]:
with open('data/hayek/pdftotext/HayekIndividualismoElverdaderoyelfalso.txt','r',encoding='utf-8') as f:
    doc = f.read()

## 2. Limpieza

Se reemplazan los encabezados por el identificador \<newpage>:

In [ ]:
def remove_header(text:str, save:bool=False):
    header_pattern = re.compile(r"\n\x0c(\d+\s+ESTUDIOS\sPÚBLICOS|FRIEDRICH A. HAYEK\s+\d+)\n{3}")
    headers = header_pattern.findall(text)
    print('* Encabezados hallados:')
    for h in headers:
        print('\t',h)
    newtext = header_pattern.sub('\n<newpage>',text)
    if save:
        with open('data/hayek/clean/1_headers_removed.txt','w') as f:
            f.write(newtext)
            f.close()
    return newtext

Se unen palabras separadas por guión y salto de línea:

In [ ]:
def join_words(text:str, verbose=True, save:bool=False):
    sep = re.compile(r"\w+(\-\n)(\s{8})?\w+")
    separated_words = sep.finditer(text)
    if verbose:
        print('* Palabras separadas por guión:')
    for sw in separated_words:
        sep_word = sw.group()
        joined_word = re.sub(r'\-\n+(\s{8})?','',sep_word)
        if verbose:
            print('\t',
                  re.sub(r'\s+','',sep_word.replace('\n','\\n')),
                  '>',
                  joined_word)
        text = re.sub(sep_word, joined_word, text)
    if save:
        with open('data/hayek/clean/2_joined_words.txt','w') as f:
            f.write(text)
            f.close()
    return text


def join_words2(text:str, verbose=True, save:bool=False):
    sep = re.compile(r"\w+(\-\n+)\w+")
    separated_words = sep.finditer(text)
    if verbose:
        print('* Palabras separadas por guión:')
    for sw in separated_words:
        sep_word = sw.group()
        joined_word = re.sub(r'\-\n+','',sep_word)
        if verbose:
            print('\t',
                  re.sub(r'\s+','',sep_word.replace('\n','\\n')),
                  '>',
                  joined_word)
        text = re.sub(sep_word, joined_word, text)
    if save:
        with open('data/hayek/clean/12_joined_words.txt','w') as f:
            f.write(text)
            f.close()
    return text

Se identifican las notas al pie con la etiqueta \<footnote>:

In [ ]:
def footnotes_identify(text:str, save=False):
    footnotes_start = re.compile(r'\s{9}(\*|\d+)\s')
    all_footnotes = footnotes_start.finditer(text)
    print('* Inicio de notas al pie localizados')
    foots_list = list()
    for af in all_footnotes:
        note = af.group()
        foots_list.append(note)
        text = text.replace(note, '\n<footnote>\n'+note)
    print('\t', list(map(str.strip,foots_list)))
    if save:
        with open('data/hayek/clean/3_footnotes_identified.txt','w') as f:
            f.write(text)
            f.close()
    return text

In [ ]:
def end_identify(text:str, save=False):
    print('* Final del documento localizado')
    text = re.sub(r'\x0c$','<text-end>',text)
    if save:
        with open('data/hayek/clean/4_text_end_identified.txt','w') as f:
            f.write(text)
            f.close()
    return text

In [ ]:
def unique_newline(text:str):
    newtext = re.sub(r'\n+','\n',text)
    return newtext

In [ ]:
def remove_newline(text:str):
    newtext = re.sub(r'\n',' ',text)
    newtext = re.sub(r'\s{2}',' ',text)
    return newtext

In [ ]:
def extract_footnote(text:str, save=False):
    footnote_paragraph = re.compile(r'''
                                    (?<=\<footnote\>) # cualquier cosa después de una marca de footnote
                                    \n\s{9}(\*|\d+)   # salto de línea + 9 espacios + * o dígitos
                                    (.*?\n)+?         # cualquier cosa (incl. salto de línea)   
                                    (?=\<)            # hasta que haya un <
                                    ''',       
                                    re.X)
    paragraphs = footnote_paragraph.finditer(text)
    footnotes = dict()
    for p in paragraphs:
        note = unique_newline(p.group().strip())
        note = remove_newline(join_words(note, verbose=False))
        note_nr = re.match(r'(\*|\d+)',note).group()
        note_text = re.sub(r'^(\*|\d+)','',note).strip()
        footnotes[note_nr] = note_text
        text = text.replace(p.group(),'<footnote-removed>')
    print('* Notas al pie extraídas')
    if save:
        with open('data/hayek/clean/5_footnotes_removed.txt','w') as f:
            f.write(text)
            f.close()
        with open('data/hayek/clean/8_footnotes_dict.json','w') as f:
            json.dump(footnotes, f, ensure_ascii=False, indent=4)
    return text, footnotes

In [ ]:
def remove_footnote_marks(text, save=False):
    text = re.sub(r'\<(footnote(\-removed)?)\>','',text)
    print('* Etiquetas de nota al pie eliminadas')
    if save:
        with open('data/hayek/clean/6_footnotes_label_removed.txt','w') as f:
            f.write(text)
            f.close()
    return text

In [ ]:
def remove_main_header(text:str, save=False):
    text = re.sub(r'\s+DOCUMENTO','',text)
    print('* Encabezado principal eliminado (DOCUMENTO)')
    if save:
        with open('data/hayek/clean/7_main_header_removed.txt','w') as f:
            f.write(text)
            f.close()
    return text

In [ ]:
def identify_notes_in_text(text: str, verbose=True, save=False):
    note_patterns = re.compile(r'([a-zA-Z]|\.)(\*|\d+)')
    note_marks = note_patterns.finditer(text)
    print('* Marcadores de nota al pie reemplazdos por etiquetas')
    for nm in note_marks:
        note = nm.group()
        nr = note[1:]
        note_identifyer = note[0]+f'<footnote-{nr}>'
        text = re.sub(re.escape(note), note_identifyer, text, count=1)
        if verbose:
            print('\t',note,' > ',note_identifyer)
    if save:
        with open('data/hayek/clean/8_footnotes_marks_identified.txt','w') as f:
            f.write(text)
            f.close()
    return text

In [ ]:
def remove_newpage(text:str, save=False):
    text = re.sub(r'\<newpage\>','',text)
    if save:
        with open('data/hayek/clean/9_newpage_removed.txt','w') as f:
            f.write(text)
            f.close()
    return text

In [ ]:
def add_indentation_tabs(text:str, save=False):
    text = re.sub(r'\n\s{8}([A-Z])',r'\n\t\1',text)
    if save:
        with open('data/hayek/clean/10_indentation_tabs.txt','w') as f:
            f.write(text)
            f.close()
    return text

In [ ]:
def remove_newlines_inside_paragraphs(text:str, save=False):
    text = re.sub(r'([A-Za-záéíóú]|\,|\.|\;|\?|\!|\))\n+(?!\t)',' ',text)
    if save:
        with open('data/hayek/clean/11_join_paragraphs.txt','w') as f:
            f.write(text)
            f.close()
    return text

In [ ]:
def remove_spaces(text:str, save=False):
    text = re.sub(' +',' ',text)
    if save:
        with open('data/hayek/clean/13_spaces_removed.txt','w') as f:
            f.write(text)
            f.close()
    return text

In [ ]:
def remove_many_newlines(text:str, save=True):
    text = re.sub(r'\n+',r'\n',text)
    if save:
        with open('data/hayek/clean/14_many_newlines_removed.txt','w') as f:
            f.write(text)
            f.close()
    return text

In [ ]:
newdoc = remove_header(doc, save=True)

* Encabezados hallados:
	 2                                                            ESTUDIOS PÚBLICOS
	 FRIEDRICH A. HAYEK                                                             3
	 4                                                               ESTUDIOS PÚBLICOS
	 FRIEDRICH A. HAYEK                                                                      5
	 6                                                                ESTUDIOS PÚBLICOS
	 FRIEDRICH A. HAYEK                                                                      7
	 8                                                                 ESTUDIOS PÚBLICOS
	 FRIEDRICH A. HAYEK                                                                       9
	 10                                                             ESTUDIOS PÚBLICOS
	 FRIEDRICH A. HAYEK                                                                      11
	 12                                                               ESTUDIOS PÚBLICOS
	 FRIEDRICH A. HAYE

In [ ]:
newdoc = join_words(newdoc, verbose=False, save=True)

In [ ]:
newdoc = footnotes_identify(newdoc, save=True)

* Inicio de notas al pie localizados
	 ['*', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32']


In [ ]:
newdoc = end_identify(newdoc, save=True)

* Final del documento localizado


In [ ]:
newdoc, footnotes = extract_footnote(newdoc, save=True)

* Notas al pie extraídas


In [ ]:
footnotes['*']

'Este ensayo corresponde a una exposición pronunciada en la duodécima\nFinlay Lecture en la University College de Dublín, en diciembre de 1945. Fue publicado\nen 1946 en Dublín y Oxford y aparece en el volumen Individualism and Economic\nOrder (The University of Chicago, 1948, reimpreso posteriormente por Gateway Editions Ltd., South Bend, Indiana). Se publica con la debida autorización.\nEstudios Públicos, 22.'

In [ ]:
newdoc = remove_footnote_marks(newdoc, save=True)

* Etiquetas de nota al pie eliminadas


In [ ]:
newdoc = remove_main_header(newdoc, save=True)

* Encabezado principal eliminado (DOCUMENTO)


In [ ]:
newdoc = identify_notes_in_text(newdoc, save=True)

* Marcadores de nota al pie reemplazdos por etiquetas
	 O*  >  O<footnote-*>
	 .1  >  .<footnote-1>
	 .2  >  .<footnote-2>
	 .3  >  .<footnote-3>
	 .4  >  .<footnote-4>
	 .5  >  .<footnote-5>
	 .6  >  .<footnote-6>
	 .7  >  .<footnote-7>
	 .9  >  .<footnote-9>
	 .10  >  .<footnote-10>
	 .11  >  .<footnote-11>
	 .12  >  .<footnote-12>
	 .13  >  .<footnote-13>
	 .15  >  .<footnote-15>
	 .17  >  .<footnote-17>
	 .19  >  .<footnote-19>
	 .20  >  .<footnote-20>
	 .22  >  .<footnote-22>
	 .23  >  .<footnote-23>
	 .24  >  .<footnote-24>
	 .26  >  .<footnote-26>
	 .27  >  .<footnote-27>
	 .28  >  .<footnote-28>
	 .29  >  .<footnote-29>
	 .30  >  .<footnote-30>
	 .32  >  .<footnote-32>


Las notas 5 y 15 están cortadas.

In [ ]:
newdoc = remove_newpage(newdoc, save=True)

In [ ]:
newdoc = add_indentation_tabs(newdoc, save=True)

In [ ]:
newdoc = remove_newlines_inside_paragraphs(newdoc, save=True)

In [ ]:
newdoc = join_words2(newdoc, save=True)

* Palabras separadas por guión:
	 principal-\n\n\n\n\nmente > principalmente
	 institucio-\n\n\n\n\nnes > instituciones
	 he-\n\ncho > hecho
	 esfuer-\n\n\n\nzo > esfuerzo
	 espontá-\n\nneas > espontáneas
	 indivi-\n\n\ndualismo > individualismo


In [ ]:
newdoc = remove_spaces(newdoc, save=True)

In [ ]:
newdoc = remove_many_newlines(newdoc, save=True)

In [ ]:
newdoc = '\n'.join([line.strip(' ') for line in newdoc.split('\n') if (line!='') and (line!=' ')])

In [ ]:
newdoc = re.sub('\s*<text-end>','.',newdoc)

In [ ]:
with open('data/hayek/clean/15_clean_text.txt','w') as f:
    f.write(newdoc)
    f.close()

In [ ]:
with open('data/hayek/clean/14_many_newlines_removed.txt','r',encoding='utf-8') as f:
    doc = f.read()
print(doc)


 INDIVIDUALISMO:
 EL VERDADERO Y EL FALSO<footnote-*>
 Friedrich A. Haye 
	Hay en este ensayo un notable esfuerzo por precisar el significad de las palabras y los conceptos. Se encontrará en seguida un sistemático y esclarecedor paralelo entre dos tradiciones contrapuesta del pensamiento individualista, de naturaleza gradualista y conservadora una y constructivista y revolucionaria la otra. Hayek proporciona una sólida fundamentación del orden libertario desde el punt de vista filosófico, económico y social, enriqueciendo una vertient intelectual que, según él, alcanzó expresiones acabadas en las obra de John Locke, Bernard Mandeville, Josiah Tucker, Adam Ferguson,
	Adam Smith, Edmund Burke, Lord Acton y Alexis de Tocqueville.
	Pieza fundamental de la renovación del pensamiento liberal, est trabajo, que fue expuesto hace casi cuarenta años, permite conoce uno de los hilos conductores de la trama intelectual de nuestro tiempo 
	FRIEDRICH A. HAYEK. Premio Nobel de Economía 1974 y Presid

In [ ]:
print(doc)